In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import requests
import re

In [2]:
import spacy
# from spacy.matcher import PhraseMatcher
# from skillNer.general_params import SKILL_DB
# from skillNer.skill_extractor_class import SkillExtractor

KeyboardInterrupt: 

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
df = pd.read_excel("D:\EY\LinkedIn-Job-Scraper\output100.xlsx")

In [ ]:
jd_col = df['Job Description']

In [ ]:
# All of NLP Preprocessing

'''
Removing of Special Characters
    |
    v
Tokenization
    |
    v
Stop Words Removal
    |
    v
Filtering & lowering
    |
    v
Joining the processed tokens
'''

def preprocess_job_description(job_description):

    # Removing Special characters & numbers
    job_description = re.sub(r'[^a-zA-Z]', ' ', job_description)

    job_description = re.sub(r'\d+\.?\d*', '',job_description)

    # Tokenization
    tokens = word_tokenize(job_description)

    # Remove Stop Words
    stop_words = set(stopwords.words('english'))

    # Lower cases all of the JD
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]

    # Making the tokens into a proper string



    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

In [ ]:
jd_col_list = jd_col.tolist()

In [ ]:
processed_list = []

In [ ]:
for element in jd_col_list:
    process = preprocess_job_description(str(element))
    processed_list.append(process)

In [ ]:
import requests
def extract_skills_huggingface(jd):
    MAX_LENGTH = 512
    # Split the input string into chunks of maximum length
    chunks = [jd[i:i+MAX_LENGTH] for i in range(0, len(jd), MAX_LENGTH)]
    skl = []
    for chunk in chunks:
        response = requests.post("https://jjzha-skill-extraction-demo.hf.space/run/predict", json={
            "data": [
                chunk
            ]
        }).json()

        data = response["data"]

        for i in data:
            for sk in i:
                if sk[1] == 'Skill':
                    skl.append(sk[0])

    return skl


In [ ]:
skill = []
for jd in processed_list:
    #print(type(jd))
    skills = extract_skills_huggingface(jd)
    skill.append(skills)


In [ ]:
skill

In [ ]:
# import openpyxl
#
# def put_list_of_lists_in_excel(list_of_lists, excel_file):
#     """
#     Puts a list of lists into different rows of an Excel file.
#
#     Args:
#         list_of_lists: The list of lists to be inserted.
#         excel_file: The path to the Excel file.
#     """
#
#     wb = openpyxl.Workbook()
#     sheet = wb.active
#
#     row = 1
#     for list_data in list_of_lists:
#         for value in list_data:
#             sheet.cell(row=row, column=1).value = value
#             row += 1
#
#     wb.save(excel_file)
#
# if __name__ == "__main__":
#     list_of_lists = skill
#     excel_file = "D:\EY\LinkedIn-Job-Scraper\output100.xlsx"
#
#     put_list_of_lists_in_excel(list_of_lists, excel_file)


In [ ]:
df['skills'] = skill

In [ ]:
# Save the DataFrame back to the Excel file
df.to_excel('D:\EY\LinkedIn-Job-Scraper\output_skills10.xlsx', index=False)